<center>
<h1>LSTM con Keras, un flujo básico pero completo</h1>


<p> Julio Waissman Vilanova </p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/LSTM-IMdb.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

<p>
Tomado parcialmente y adaptado de varias libretas de la documentación de Keras
</p>


</center>

In [1]:
import re
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Obteniendo datos

Vamos a recuperar la base de datos globera de IMdb que se usa para probar casi todos los modelos. Vamos a recuperar los adatos de 

``https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz``

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  3 80.2M    3 2592k    0     0  1278k      0  0:01:04  0:00:02  0:01:02 1281k
  9 80.2M    9 8096k    0     0  2675k      0  0:00:30  0:00:03  0:00:27 2679k
 13 80.2M   13 10.9M    0     0  2751k      0  0:00:29  0:00:04  0:00:25 2754k
 16 80.2M   16 13.6M    0     0  2780k      0  0:00:29  0:00:05  0:00:24 2787k
 21 80.2M   21 17.3M    0     0  2960k      0  0:00:27  0:00:06  0:00:21 3626k
 25 80.2M   25 20.7M    0     0  3024k      0  0:00:27  0:00:07  0:00:20 3734k
 29 80.2M   29 23.8M    0     0  3043k      0  0:00:26  0:00:08  0:00:18 3267k
 33 80.2M   33 27.0M    0     0  3061k      0  0:00

 y vamos a investigas la estructura y lo que hay...

In [ ]:
!ls aclImdb

In [ ]:
!ls aclImdb/test

In [ ]:
!ls aclImdb/train

In [ ]:
!cat aclImdb/train/pos/6248_7.txt

Solo nos interesan las evaluaciones positivas y negativas (para hacer una simple clasificación binaria y simplificar la aplicación), por lo que vamos a borrar el folder `unsup`:

In [ ]:
!rm -r aclImdb/train/unsup

Ahora si, vamos a usar las librerías de `Keras` para leer los datos usando [`keras.utils.text_dataset_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/text_dataset_from_directory).

En este momento es donde tenemos que determinar el tamaño de los lotes.

In [ ]:
batch_size = 32         # Tamaño de los minibatches

raw_train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

In [ ]:
print(f"Numero de batches en raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Numero de batches en raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Numero de batches en raw_test_ds: {raw_test_ds.cardinality()}")

Es importante revisar los datos crudos para tener una idea de como se recuperaron y cual es la forma que tienen.

Esto lo podemos hacer tomando algunos datos de cada batch e imprimiendolos:

In [ ]:
import textwrap

for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(textwrap.fill(text_batch.numpy()[i], 80, subsequent_indent='> '))
        print("\ntarget =", label_batch.numpy()[i])

## Preparando los datos

Vamos ahora a convertir cada string de datos en una serie de índices numéricos, los cuales puedan entrar en 
un modelo neuronal. Para esto, vamos a generar índices a partir de las palabras existentesd en el texto.

Este métdo puede ser no el mejor, ya que el vocabulario se fija en relación al vocabulario encontrado en el
conjunto de aprendizaje. Más adelante veremos mñetodos más sofisticados para hacer la indezación, o como
usar un vocabulario indexado ya preestablecido.

Por el momento vamos primero a especificar el proceso de limpieza de texto (preprocesamiento) el cual será muy sencillo para este ejemplo y consiste en:

1. Convertir a minúsculas todas las letras
2. Eliminar los saltos de linea en formato *html* ( `<br /> `) 
3. Eliminar los signos de puntuación

Igualmente, vamos a generar los minibatches con secuencias de `sequence_length` palabras. Esto es, si es insuficiente, se trunca el texto y si es 
demasiado, se completa el texto con 0's. De esa manera, todos los modelos aprenden con secuencias del mismo tamaño.

Se utilizan hasta `max_features` tokens diferentes. De haber más, estos se eliminan en función de su frecuencia.

Para esto vamos a utilizar la capa de `Keras` de [`layers.TextVectorization`](https://keras.io/api/layers/preprocessing_layers/text/text_vectorization/)

In [ ]:
# Model constants.
max_features = 20000
sequence_length = 500

# Preprocesamiento
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Capa de vectorización (encontrar los índices por palabra)
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vectorize_layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)

# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [ ]:
# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [ ]:
print("Donde se guardan los datos de entrenamiento")
print("train_ds.cardinality() = ", train_ds.cardinality())

ejemplo = train_ds.take(1)

print("\nY un minibatch se representa de esta manera: \n")
print(ejemplo.get_single_element())

## Modelo basado en LSTM multicapa

Vamos a hacer un modelo multicapa, el cual seguramente requerirá de ajustes de su parte.

Vamos a utilizar la forma funcional de definir un modelo neuronal:

In [ ]:
emb = 128               # Embedding size
unidades = 128          # Hidden units per layer


# Entrada en indices
inputs = keras.Input(shape=(None,), dtype="int64")

# Capa de embeddings
x = layers.Embedding(max_features, emb)(inputs)

# Dos capas de LSTMs
x = layers.LSTM(unidades, return_sequences=True)(x)
x = layers.LSTM(unidades)(x)

# Salida
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)
model.summary()

Compilamos y ponemos a aprender el modelo (usando BPTT en forma automñatica)

In [ ]:
model.compile(
    "adam",
    "binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Y probamos con los datos de test

In [ ]:
model.evaluate(test_ds)

Y ahora vamos a probar con bi-LSTM, haciendo un poco más complicado (aunque no mucho) el código

In [ ]:
emb = 128
unidades = 128

# Input 
inputs = keras.Input(shape=(None,), dtype="int32")

# Capa de embeddings
x = layers.Embedding(max_features, emb)(inputs)

# bi-LSTMs
x = layers.Bidirectional(
    layers.LSTM(unidades, return_sequences=True)
)(x)
x = layers.Bidirectional(
    layers.LSTM(unidades)
)(x)

# Vanilla hidden layer:
x = layers.Dense(unidades, activation="relu")(x)

# Salida
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model_bi = keras.Model(inputs, predictions)
model_bi.summary()

In [ ]:
model_bi.compile(
    "adam",
    "binary_crossentropy",
    metrics=["accuracy"]
)

model_bi.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

In [ ]:
model_bi.evaluate(test_ds)

## Modelo por convolucionales de 1 dimensión

Este modelo viene como modelo de base en Keras, y es un buen inicio para ver como usar convolucionales como modelos para PLN.



In [ ]:
emb = 128
unidades = 128
ventana = 7
drop= 0.5

# Entrada
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Capa de embeddings
x = layers.Embedding(max_features, emb)(inputs)
x = layers.Dropout(drop)(x)

# Conv1D + global max pooling
x = layers.Conv1D(
    unidades, 
    ventana, 
    padding="valid", 
    activation="relu", 
    strides=3
)(x)
x = layers.Conv1D(
    unidades, 
    ventana, 
    padding="valid", 
    activation="relu", 
    strides=3
)(x)
x = layers.GlobalMaxPooling1D()(x)

# Vanilla hidden layer:
x = layers.Dense(unidades, activation="relu")(x)
x = layers.Dropout(drop)(x)

# Salida
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model_conv1d = tf.keras.Model(inputs, predictions)

In [ ]:
# Compile the model with binary crossentropy loss and an adam optimizer.
model_conv1d.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=3
)

In [ ]:
model_conv1d.evaluate(test_ds)

## Modelo para producción

Si ya tenemos nuestro modelo funcionando, y nos gusta, y queremos dejarlo en un formato que permita aplicarlo a los datos en crudo, es necesario empaquetar todo nuestro procedimiento en un solo procedimiento de principio a fin. 

Agregamos aqui el truco para empaqetar todo, cuando ya no se espera reentrenar el modelo (al menos no en el corto plazo).

In [ ]:
modelo_seleccionado = model_bi

# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")

# Turn strings into vocab indices
indices = vectorize_layer(inputs)

# Turn vocab indices into predictions
outputs = modelo_seleccionado(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)

end_to_end_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

end_to_end_model.save('nombre_codigo.keras')

In [ ]:
end_to_end_model = keras.saving.load_model("nombre_codigo.keras")

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)